<a href="https://colab.research.google.com/github/imaj1512/softronics/blob/main/face_and_hand_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implement real-time face and hand detection and integrate it with a Flask web application for online streaming?

libraries

In [1]:
!pip install opencv-python mediapipe flask

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 32.3 MB/s eta 0:00:00


In [ ]:
from flask import Flask, render_template, Response
import cv2
import mediapipe as mp

app = Flask(__name__)

mp_face_detection = mp.solutions.face_detection
mp_hands = mp.solutions.hands

face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)
hands = mp_hands.Hands(min_detection_confidence=0.5)

camera = cv2.VideoCapture(0)  # Use default camera

def gen_frames():
    while True:
        success, frame = camera.read()
        if not success:
            break
        else:
            # Convert the BGR image to RGB.
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Process the image and find faces and hands.
            results_face = face_detection.process(image)
            results_hands = hands.process(image)

            # Draw the face and hand detections on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results_face.detections:
                for detection in results_face.detections:
                    mp_face_detection.draw_detection(image, detection)

            if results_hands.multi_hand_landmarks:
                for hand_landmarks in results_hands.multi_hand_landmarks:
                    mp_hands.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style())

            ret, buffer = cv2.imencode('.jpg', image)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


Create  HTML template

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Real-time Face and Hand Detection</title>
</head>
<body>
    <h1>Real-time Face and Hand Detection</h1>
    <img src="{{ url_for('video_feed') }}" width="640" height="480">
</body>
</html>

Run flask application

In [ ]:
python app.py